# Setting UP SNPE SDK

In [ ]:
import os
os.environ['QNN_SDK_ROOT']="/local/mnt/workspace/gokul/QNN/2.19.0.240124" #set up your qnn path here.
os.environ['ANDROID_NDK_ROOT']="/local/mnt/workspace/gokul/android-ndk-r25c"
os.environ['RAW_FILE_FOLDER']="input/raw"
os.environ['MODEL_NAME']="fcn_resnet101_fp32"
os.environ['QUANTIZED_MODEL_NAME']="fcn_resnet101_quant16_w8a16"
os.environ['MODEL_PATH']="models/fcn_resnet101_fp32"
os.environ['QUANTIZED_MODEL_PATH']="models/fcn_resnet101_quant16_w8a16"
os.environ['TARGET_INPUT_LIST']="input/input.txt"
os.environ['ONDEVICE_FOLDER']="fcn_resnet101"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="bc468c1d" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['QNN_TARGET_ARCH']="aarch64-android"
os.environ['QNN_TARGET_STL']="libQnnHtp.so"

In [ ]:
import os
import torchvision.models.segmentation as models
import torch
import torch.nn as nn

In [ ]:
pretrained_model = models.fcn_resnet101(pretrained=True)
class CustomModel(nn.Module):
    def __init__(self,pretrained_model):
        super(CustomModel,self).__init__()
        self.pretrained_model = pretrained_model
        self.argmax = nn.LogSoftmax(dim=1)
    def forward(self,x):
        output_dict = self.pretrained_model(x)
        output = output_dict['out']
        argmax_output = torch.argmax(output,dim=1,keepdim=False)
        return argmax_output
model = CustomModel(pretrained_model)
input = torch.randn(1,3,400,400)
output = model(input)
output.shape

In [ ]:
os.makedirs('models',exist_ok=True)

In [ ]:
dummy_input = torch.randn(1,3, 400, 400).type(torch.FloatTensor).to('cpu')
torch.onnx.export(model, dummy_input, "./models/fcn_resnet101.onnx",opset_version=11)

In [ ]:
import os
import cv2
import glob
import numpy as np
import torch
from os.path import isfile, join
import matplotlib.pyplot as plt 
from PIL import Image
from torchvision import transforms as T

In [ ]:
os.makedirs('input',exist_ok=True)
os.makedirs('input/dataset',exist_ok=True)

## Download dataset in input/dataset/ directory
#### Try in terminal if notebook getting freezed

In [ ]:
%%bash
cd input/dataset/
wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
tar -xvf VOCtrainval_11-May-2012.tar

In [ ]:
from tqdm import tqdm
import requests
url = 'http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar'
response = requests.get(url,stream=True)
size = int(response.headers.get('content-length',0))
with open('input/dataset/VOCtrainval_11-May-2012.tar','wb') as file:
    for data in tqdm(response.iter_content(chunk_size=1024), total=size, unit='B', unit_scale=True):
        file.write(data)

In [ ]:
from tqdm import tqdm
import tarfile
# Define the path to the large file
file_path = "input/dataset/VOCtrainval_11-May-2012.tar"
with tarfile.open(file_path, "r:tar") as tar:
    file_size = sum([member.size for member in tar.getmembers()])   
    progress_bar = tqdm(total=file_size, unit="B", unit_scale=True) 
    for member in tar.getmembers():
        tar.extract(member)
        progress_bar.update(member.size)  
    progress_bar.close()

In [ ]:
%%bash
cp -r VOCdevkit/ input/dataset/
rm -rf VOCdevkit/

In [ ]:
def preProcess(file):
    img = Image.open(file).convert('RGB')
    img = transform(img).unsqueeze(0) # To tensor of NCHW
    # print(img.shape)
    img = np.transpose(img, (0,2,3,1))
    #print(img.shape)  
    img = img.numpy()
    print(img.shape)
    return img

In [ ]:
os.makedirs('input/raw',exist_ok=True)

In [ ]:
test_images_dir = "input/dataset/VOCdevkit/VOC2012/JPEGImages/"
image_count=15

transform = T.Compose([
                T.Resize((400,400)),
                
                T.ToTensor(),
                T.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225]),
            ])

image_paths = []
c=0
inputlist = open('input/dataset/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt', 'r')
Lines = inputlist.readlines()
Lines = [line.strip() + ".jpg" for line in Lines]
for line in Lines:
    image_paths.extend(glob.glob(os.path.join(test_images_dir, line)))
    c = c+1
    if(c==image_count):
        break
image_paths = sorted(image_paths)
count=0
for img_path in image_paths:
    print("img_path: ", img_path)
    img = preProcess(img_path)
    name = img_path.replace(test_images_dir,"")
    name = name.split('.')[0]
    #print(name)
    raw_data = os.path.join('input/raw/preproc_'+name+'.raw')
    img.tofile(raw_data)
    count = count+1
    if count==image_count:
        break
    print("raw image saved")

In [ ]:
directory_path = 'input/raw/'
output_file_path = 'input/input.txt'  # The file where the output will be saved
all_files = os.listdir(directory_path)
# Filter only the .raw files and create a list of their names
raw_files = [file for file in all_files if file.endswith('.raw')]
raw_files = sorted(raw_files)
# Write the file names to the output file
with open(output_file_path, 'w') as f:
    c=0
    for raw_file in raw_files:
        f.write(f"./raw/{raw_file}\n")
        c=c+1
print(f"File names written to {output_file_path}.")

In [ ]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
export TMPDIR=${PWD}/temp
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network models/fcn_resnet101.onnx --output_path ${MODEL_PATH}.cpp
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${MODEL_PATH}.cpp -b ${MODEL_PATH}.bin -o models/model_libs
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs/x86_64-linux-clang/lib${MODEL_NAME}.so \
              --binary_file ${MODEL_NAME}.serialized

In [ ]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
export TMPDIR=${PWD}/temp
cd input
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network ../models/fcn_resnet101.onnx --output_path ../${QUANTIZED_MODEL_PATH}.cpp --input_list input.txt \
--param_quantizer "adjusted" --act_quantizer "enhanced"
cd ..
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${QUANTIZED_MODEL_PATH}.cpp -b ${QUANTIZED_MODEL_PATH}.bin -o models/model_libs2
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs2/x86_64-linux-clang/lib${QUANTIZED_MODEL_NAME}.so \
              --binary_file ${QUANTIZED_MODEL_NAME}.serialized

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/dsp/lib"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/$QNN_TARGET_STL /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/bin/$QNN_TARGET_ARCH/qnn-net-run /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/hexagon-v73/unsigned/*.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/*.so /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
#find ./raw -name *.raw > list.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push output/${QUANTIZED_MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push output/${MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push models/model_libs2/aarch64-android/lib${QUANTIZED_MODEL_NAME}.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="fcn_resnet101"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="fcn_resnet101_quant16_w8a16"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_Quant_DSP
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnHtp.so --input_list input.txt --retrieve_context ${QUANTIZED_MODEL_NAME}.serialized.bin --output_dir $OUTPUT_FOLDER

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="fcn_resnet101"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="fcn_resnet101_quant16_w8a16"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_CPU
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnCpu.so --input_list input.txt --model lib${QUANTIZED_MODEL_NAME}.so --output_dir $OUTPUT_FOLDER

In [ ]:
os.makedirs('output', exist_ok=True)

In [ ]:
%%bash
rm -rf OUTPUT_Quant_DSP
rm -rf OUTPUT_CPU

In [ ]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_Quant_DSP output/OUTPUT_Quant_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_CPU output/OUTPUT_CPU

In [ ]:
def image_overlay(image, segmented_image):
    alpha = 1 # transparency for the original image
    beta = 0.8 # transparency for the segmentation map
    gamma = 0 # scalar added to each sum
    # print(image.size)
    # print(segmented_image.shape)
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2BGR)
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.addWeighted(image, alpha, segmented_image, beta, gamma, image)
    return image

In [ ]:
import torchvision.transforms as transforms
crop_size = 400
transform = transforms.Compose([
    transforms.Resize((crop_size,crop_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
label_map = [
               (0, 0, 0),  # background
               (128, 0, 0), # aeroplane
               (0, 128, 0), # bicycle
               (128, 128, 0), # bird
               (0, 0, 128), # boat
               (128, 0, 128), # bottle
               (0, 128, 128), # bus 
               (128, 128, 128), # car
               (64, 0, 0), # cat
               (192, 0, 0), # chair
               (64, 128, 0), # cow
               (192, 128, 0), # dining table
               (64, 0, 128), # dog
               (192, 0, 128), # horse
               (64, 128, 128), # motorbike
               (192, 128, 128), # person
               (0, 64, 0), # potted plant
               (128, 64, 0), # sheep
               (0, 192, 0), # sofa
               (128, 192, 0), # train
               (0, 64, 128) # tv/monitor
]

In [ ]:
def get_segment_labels(image, model, device):
    # transform the image to tensor and load into computation device
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)
    return outputs

In [ ]:
def draw_segmentation_map(outputs):
    labels = outputs.detach().cpu().numpy()
    # create Numpy arrays containing zeros
    # later to be used to fill them with respective red, green, and blue pixels
    red_map = np.zeros_like(labels).astype(np.uint8)
    green_map = np.zeros_like(labels).astype(np.uint8)
    blue_map = np.zeros_like(labels).astype(np.uint8)
    
    for label_num in range(0, len(label_map)):
        index = labels == label_num
        red_map[index] = np.array(label_map)[label_num, 0]
        green_map[index] = np.array(label_map)[label_num, 1]
        blue_map[index] = np.array(label_map)[label_num, 2]
        
    segmentation_map = np.stack([red_map, green_map, blue_map], axis=2)
    return segmentation_map

In [ ]:
def ImageNames():
    inputlist = open('input/input.txt', 'r')
    Lines = inputlist.readlines()
    count = 0
    imageList = []
    for line in Lines:
        name = line.split("preproc_",1)[1]
        name = name.split('.')[0]
        imageList.append(name)
        count += 1
    return imageList
imageList = ImageNames()
print((imageList))

In [ ]:
import torch
import numpy as np
device = torch.device('cpu')
model.eval()

In [ ]:
os.makedirs('output/model_prediction', exist_ok=True)

In [ ]:
image_dir = 'input/dataset/VOCdevkit/VOC2012/JPEGImages/'
from PIL import Image
import cv2
for i in range(0,len(imageList)):

    image = Image.open(image_dir+imageList[i]+'.jpg')
    # do forward pass and get the output dictionary
    image  = image.resize((crop_size,crop_size))
    # print(image.size)
    outputs = get_segment_labels(image, model, device)
    # get the data from the `out` key
    # outputs = outputs['out']
    # print(type(outputs))
    # print(outputs.shape)
    segmented_image = draw_segmentation_map(outputs[0])
    print(image.size)
    print(segmented_image.shape)
    final_image = image_overlay(image, segmented_image)
    # show the segmented image and save to disk
    # cv2.imshow('Segmented image', final_image)
    # cv2.waitKey(0)
    cv2.imwrite(f"output/model_prediction/{imageList[i]}.jpg", final_image)

In [ ]:
def PostProc(img_path, out_path,i):
    res = np.fromfile(img_path, dtype="float32")
    res_reshape = res.reshape((1,400,400)).astype(np.float32)
    model_img = torch.from_numpy(res_reshape)
    segmented_image = draw_segmentation_map(model_img[0])
    image = Image.open(image_dir+imageList[i]+'.jpg')
    # do forward pass and get the output dictionary
    image  = image.resize((crop_size,crop_size))
    final_image = image_overlay(image, segmented_image)
    cv2.imwrite(f"{out_path}", final_image)

In [ ]:
os.makedirs('output/test_results', exist_ok=True)
os.makedirs('output/test_results/32b_arm', exist_ok=True)
os.makedirs('output/test_results/8b_dsp', exist_ok=True)

In [ ]:
test_images_dir = "output/OUTPUT_32b_CPU/"
image_dir = 'input/dataset/VOCdevkit/VOC2012/JPEGImages/'
import cv2
import os
import numpy as np
import torch
from PIL import Image
for i in range(0,len(imageList)):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/1002.raw'
    out_path = 'output/test_results/32b_arm/'+imageList[i]+'_prediction_32b_arm.png'
    PostProc(img_path, out_path,i)
    i = i +1

In [ ]:
test_images_dir = "output/OUTPUT_8b_DSP/"
for i in range(0,len(imageList)):
    img_path = os.path.join(test_images_dir, 'Result_')
    img_path = img_path+str(i)+'/1002.raw'
    out_path = 'output/test_results/8b_dsp/'+imageList[i]+'_prediction_8b_dsp.png'
    PostProc(img_path, out_path,i)
    i = i +1

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30, 100));
import cv2
for i in range(0, 5):
    
    original = cv2.imread('input/dataset/VOCdevkit/VOC2012/JPEGImages/'+imageList[i]+'.jpg')
    original = cv2.resize(original, (400,400))
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    ax = fig.add_subplot(28,4,4*i+1);
    plt.imshow(original,cmap='gray');
    ax.set_title('original image\n');
    ax.axis('off');
    
    pth_inf = cv2.imread('output/model_prediction/'+imageList[i]+'.jpg')
    pth_inf = cv2.resize(pth_inf, (400,400))
    # pth_inf = cv2.cvtColor(pth_inf, cv2.COLOR_BGR2RGB)
    # pth_overlay = image_overlay(original, pth_inf)
    ax = fig.add_subplot(28,4,4*i+2);
    plt.imshow(pth_inf,cmap='gray');
    ax.set_title('pth output\n');
    ax.axis('off');


    arm_fp32= cv2.imread('output/test_results/32b_arm/'+imageList[i]+'_prediction_32b_arm.png')
    # arm_fp32 = cv2.cvtColor(arm_fp32, cv2.COLOR_BGR2RGB)
    # fp32_overlay = image_overlay(original, arm_fp32)
    arm_fp32 = cv2.resize(arm_fp32, (400,400))
    ax = fig.add_subplot(28,4,4*i+3);
    plt.imshow(arm_fp32,cmap='gray');
    ax.set_title('fp32 on ARM\n');
    ax.axis('off');

    dsp= cv2.imread('output/test_results/8b_dsp/'+imageList[i]+'_prediction_8b_dsp.png')
    # dsp_int8 = cv2.cvtColor(dsp_int8, cv2.COLOR_BGR2RGB)
    # int8_overlay = image_overlay(original, dsp_int8)
    # print(dsp)
    ax = fig.add_subplot(28,4,4*i+4);
    plt.imshow(dsp,cmap='gray');
    ax.set_title('int8 on DSP\n');
    ax.axis('off');